In [ ]:
# Taken from: https://github.com/skrhakv/CryptoBench/blob/master/tutorial/tutorial.ipynb
import json
from typing import TypeAlias, List
BindingSite: TypeAlias = List[str]
JSON: TypeAlias = dict[str, "JSON"] | list["JSON"] | str | int | float | bool | None

def load_dataset(path: str) -> JSON:
    """Loads dataset from JSON file.

    Args:
        path (str): Path to JSON file

    Returns:
        JSON: dataset
    """
    with open(path) as f:
        dataset = json.load(f)
    return dataset


def are_binding_sites_promiscuous(binding_site1: BindingSite, binding_site2: BindingSite, lower_threshold=0.25, upper_threshold=0.5) -> bool:
    overlap_length = len(list(set(binding_site1) & set(binding_site2)))
    if overlap_length / len(binding_site1) < upper_threshold and overlap_length / len(binding_site2) < upper_threshold and \
        overlap_length / len(binding_site1) > lower_threshold and overlap_length / len(binding_site2) > lower_threshold:
        return True
    return False



In [ ]:
CRYPTOBENCH_PATH = '../../tutorial/cryptobench'
dataset = load_dataset(f'{CRYPTOBENCH_PATH}/cryptobench-dataset/dataset.json')

for apo_pdb_id, holo_structures in dataset.items():
    for i, holo_structure1 in enumerate(holo_structures):
        for ii, holo_structure2 in enumerate(holo_structures):
            if i <= ii:
                continue
            if are_binding_sites_promiscuous(holo_structure1['apo_pocket_selection'], holo_structure2['apo_pocket_selection']) and holo_structure1['ligand'] != holo_structure2['ligand']:
                with open(f'overlapping-pockets/{holo_structure1["holo_pdb_id"]}{holo_structure1["holo_chain"]}-{holo_structure2["holo_pdb_id"]}{holo_structure2["holo_chain"]}.pml', 'w') as f:
                    f.write(f'''reinitialize
fetch {holo_structure1["holo_pdb_id"]}{holo_structure1["holo_chain"]}
fetch {holo_structure2["holo_pdb_id"]}{holo_structure2["holo_chain"]}
color blue, {holo_structure1["holo_pdb_id"]}{holo_structure1["holo_chain"]}
color aquamarine, {holo_structure1["holo_pdb_id"]}{holo_structure1["holo_chain"]} and ( (chain {holo_structure1["holo_chain"]} and resi {"+".join([i.split("_")[1] for i in holo_structure1["holo_pocket_selection"]])}) )
color forest, {holo_structure2["holo_pdb_id"]}{holo_structure2["holo_chain"]}
color yellow, {holo_structure2["holo_pdb_id"]}{holo_structure2["holo_chain"]} and ( (chain {holo_structure2["holo_chain"]} and resi {"+".join([i.split("_")[1] for i in holo_structure2["holo_pocket_selection"]])}) )
show sticks, {holo_structure1["holo_pdb_id"]}{holo_structure1["holo_chain"]} and ( (chain {holo_structure1["holo_chain"]} and resi {"+".join([i.split("_")[1] for i in holo_structure1["holo_pocket_selection"]])}) )
show sticks, {holo_structure2["holo_pdb_id"]}{holo_structure2["holo_chain"]} and ( (chain {holo_structure2["holo_chain"]} and resi {"+".join([i.split("_")[1] for i in holo_structure2["holo_pocket_selection"]])}) )
align {holo_structure1["holo_pdb_id"]}{holo_structure1["holo_chain"]}, {holo_structure2["holo_pdb_id"]}{holo_structure2["holo_chain"]}
zoom {holo_structure1["holo_pdb_id"]}{holo_structure1["holo_chain"]}''')
                print(f'Promiscuous binding site found in {apo_pdb_id} between {holo_structure1["holo_pdb_id"]}{holo_structure1["holo_chain"]} and {holo_structure2["holo_pdb_id"]}{holo_structure2["holo_chain"]}')
                print(f'Binding site 1: {holo_structure1["holo_pocket_selection"]}')
                print(f'Binding site 2: {holo_structure2["holo_pocket_selection"]}\n')
                break